In [7]:
# read in pkl file
import os
import dill as pickle

# picklefile is the location of the pickled file generated using preader 

# below could be use to import relative to current directory if easier
# cwd = os.getcwd()
# picklefile = cwd + '\\Data\\rdx3.pickle'
picklefile = 'C:\\Users\\Calum\\Documents\\Coding\\Python\\PDielec_Dev\\PDielec\\IVDWFiles\\D3\\MULTIOUTCAR\\D3_cat.pickle'
# This opens the pickled file generated by preader, 
# should read in multiple files if preader prcoessed multiple output files at once 
readers = []   
with open(picklefile,'rb') as f:
     readers = pickle.load(f)

In [8]:
# This reads in information from PDielec and turns it into struct (pymatgen) and atoms (ASE) formats
import PDielec.UnitCell as UnitCell
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter


struct_start = Structure(readers.unit_cells[0].lattice, readers.unit_cells[0].element_names, readers.unit_cells[0].fractional_coordinates)
atoms_start  = AseAtomsAdaptor.get_atoms(struct_start)
struct_end = Structure(readers.unit_cells[-1].lattice, readers.unit_cells[-1].element_names, readers.unit_cells[-1].fractional_coordinates)
atoms_end  = AseAtomsAdaptor.get_atoms(struct_end)

In [18]:
#creates set of cif files of different volumes for QHA calculation

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np


# this creates a list of percentage volume change values you want, in this case 97 to 106 (note not 107) with 1 % step
scale = np.arange(101.5, 102.5, 0.02).tolist()


# sanity check to make sure your going to make atleast 5 structures
if len(scale) < 3:
    print("you need more than 5 unit cells")
else:
    for i in range(len(scale)):
        # ScaleF is thescaling factor the lattice matric needs to be multiplied by to change the volume by the scale percentage
        ScaleF = (scale[i] / 100) ** (1/3)
        newcell = readers.unit_cells[-1].lattice * ScaleF
        # creates new struct
        struct_new = Structure(newcell, readers.unit_cells[-1].element_names, readers.unit_cells[-1].fractional_coordinates)
        # space group analysis that checks symmetry and makes sure primitive cell is formed, could do some printing/checking to confirm
        syms=SpacegroupAnalyzer(struct_new)
        struct_new = syms.get_primitive_standard_structure()
        #create list of cif filenames
        iter =+ i+1
        name = "Vol-"+ str(iter).zfill(3)
        filename = "%s.cif" % name
        #write symmetrised cif file out into cwd
        writer = CifWriter(struct_new, symprec=0.05)
        writer.write_file(filename)

# just for referance this will print out the space group symbol which maybe useful as a check  
# syms=SpacegroupAnalyzer(struct_end)
# spg_symbol=syms.get_space_group_symbol()
# print(spg_symbol)

In [10]:
#displays starting and final structure to make sure they are different

display(struct_new)
display(struct_end)

Structure Summary
Lattice
    abc : 4.760688388110572 21.88534666387562 7.856725242101092
 angles : 90.0 105.41827881381565 90.0
 volume : 789.1264075988603
      A : 0.0 4.760688388110572 0.0
      B : 21.88534666387562 0.0 0.0
      C : 0.0 -2.088817846103899 -7.573966697554613
PeriodicSite: H (0.7643, 2.4599, -1.7765) [0.6196, 0.0349, 0.2346]
PeriodicSite: H (11.7070, 0.2120, -5.7975) [0.3804, 0.5349, 0.7654]
PeriodicSite: H (20.7782, -0.0201, -2.7467) [0.1549, 0.9494, 0.3626]
PeriodicSite: H (9.8355, 2.6920, -4.8273) [0.8451, 0.4494, 0.6374]
PeriodicSite: H (21.6504, -0.5846, -4.5781) [0.1424, 0.9893, 0.6044]
PeriodicSite: H (10.7077, 3.2565, -2.9959) [0.8576, 0.4893, 0.3956]
PeriodicSite: H (20.1049, 1.7314, -2.4292) [0.5044, 0.9186, 0.3207]
PeriodicSite: H (9.1622, 0.9405, -5.1448) [0.4956, 0.4186, 0.6793]
PeriodicSite: H (18.2235, 2.2610, -3.3962) [0.6717, 0.8327, 0.4484]
PeriodicSite: H (7.2808, 0.4109, -4.1778) [0.3283, 0.3327, 0.5516]
PeriodicSite: H (18.6433, 2.3742, -0.5748

Structure Summary
Lattice
    abc : 4.6913480975584125 21.566582617 7.742290612765481
 angles : 90.0 105.41827881381566 90.0
 volume : 755.1448876544121
      A : 4.69133946 0.0 0.009002417
      B : 0.0 21.566582617 0.0
      C : -2.072712345 0.0 7.459686821
PeriodicSite: H (0.1980, 20.8134, 5.7134) [0.3804, 0.9651, 0.7654]
PeriodicSite: H (2.4207, 10.0301, 1.7553) [0.6196, 0.4651, 0.2346]
PeriodicSite: H (2.6436, 1.0910, 4.7621) [0.8451, 0.0506, 0.6374]
PeriodicSite: H (-0.0250, 11.8743, 2.7066) [0.1549, 0.5506, 0.3626]
PeriodicSite: H (3.2034, 0.2315, 2.9584) [0.8576, 0.0107, 0.3956]
PeriodicSite: H (-0.5847, 11.0148, 4.5103) [0.1424, 0.5107, 0.6044]
PeriodicSite: H (0.9171, 1.7545, 5.0716) [0.4956, 0.0814, 0.6793]
PeriodicSite: H (1.7015, 12.5378, 2.3971) [0.5044, 0.5814, 0.3207]
PeriodicSite: H (0.3970, 3.6085, 4.1177) [0.3283, 0.1673, 0.5516]
PeriodicSite: H (2.2216, 14.3918, 3.3510) [0.6717, 0.6673, 0.4484]
PeriodicSite: H (0.2801, 3.1948, 6.8978) [0.4680, 0.1481, 0.9241]
Period